In [3]:
import numpy as np
import kagglehub
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray
from keras.applications import ResNet50
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import to_categorical
import cv2

np.random.seed(42)
random.seed(42)

path = kagglehub.dataset_download("ankit1743/skyview-an-aerial-landscape-dataset")
DATASET_PATH = os.path.join(path, 'Aerial_Landscapes')
IMAGE_SIZE = (128, 128)
LBP_RADIUS = 3
LBP_N_POINTS = 8 * LBP_RADIUS

def load_dataset():
    images = []
    labels = []
    for label in os.listdir(DATASET_PATH):
        class_dir = os.path.join(DATASET_PATH, label)
        for img_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, IMAGE_SIZE)
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

def extract_lbp_features(images):
    features = []
    for img in images:
        gray = rgb2gray(img)
        lbp = local_binary_pattern(gray, LBP_N_POINTS, LBP_RADIUS, method='uniform')
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, LBP_N_POINTS + 3), range=(0, LBP_N_POINTS + 2))
        hist = hist.astype('float') / hist.sum()
        features.append(hist)
    return np.array(features)

if __name__ == "__main__":
    print("Loading dataset...")
    X, y = load_dataset()
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, stratify=y_encoded)

    print("Extracting LBP features...")
    X_train_lbp = extract_lbp_features(X_train)
    X_test_lbp = extract_lbp_features(X_test)

    print("Training SVM classifier...")
    svm = SVC(kernel='rbf', C=10, gamma='scale')
    svm.fit(X_train_lbp, y_train)
    y_pred_svm = svm.predict(X_test_lbp)
    print("SVM Classification Report:")
    print(classification_report(y_test, y_pred_svm, target_names=le.classes_))

Loading dataset...
Extracting LBP features...
Training SVM classifier...
SVM Classification Report:
              precision    recall  f1-score   support

 Agriculture       0.32      0.24      0.27       160
     Airport       0.42      0.34      0.38       160
       Beach       0.60      0.53      0.56       160
        City       0.40      0.39      0.39       160
      Desert       0.62      0.19      0.30       160
      Forest       0.53      0.57      0.55       160
   Grassland       0.58      0.56      0.57       160
     Highway       0.40      0.48      0.44       160
        Lake       0.50      0.36      0.41       160
    Mountain       0.38      0.70      0.50       160
     Parking       0.48      0.65      0.55       160
        Port       0.62      0.42      0.50       160
     Railway       0.47      0.53      0.50       160
 Residential       0.47      0.68      0.55       160
       River       0.22      0.20      0.21       160

    accuracy                      